In [ ]:
%pip install pymupdf
%pip install frontend
%pip install pillow

In [ ]:
scaleX=105
scaleY=110


In [ ]:
scaleX=108
scaleY=108

from PIL import Image, ImageDraw
from io import BytesIO
import fitz

pdf_dir = "./Data Extraction/Train Files/1040_file_2.pdf" 
doc = fitz.open(pdf_dir)
page = doc[0]

blocks = page.get_text("dict")['blocks']
text_blocks = [block for block in blocks if block['type'] == 0]
color = 'red'
width = 2
TARGET_DPI = 300
mat = fitz.Matrix(TARGET_DPI / scaleX, TARGET_DPI / scaleY)
pix_map = doc[0].get_pixmap(matrix=mat)
image = Image.open(BytesIO(pix_map.tobytes()))
draw = ImageDraw.Draw(image)

datas = [
    {"title": "Basic Information___First Name", "coordinate": [100, 325, 650, 365]},
    {"title": "Basic Information___Last Name", "coordinate": [675, 325, 1335, 365]},
    {"title": "Income___One_A", "coordinate": [1400, 1015, 1600, 1049]},
    {"title": "Income___One_B", "coordinate": [1400, 1049, 1600, 1083]},
    {"title": "Income___One_C", "coordinate": [1400, 1083, 1600, 1117]},
    {"title": "Income___One_D", "coordinate": [1400, 1112, 1600, 1150]},
    {"title": "Income___One_E", "coordinate": [1400, 1150, 1600, 1184]},
    {"title": "Income___One_F", "coordinate": [1400, 1180, 1600, 1219]},
    {"title": "Income___One_G", "coordinate": [1400, 1217, 1600, 1250]},
    {"title": "Income___One_H", "coordinate": [1400, 1250, 1600, 1284]},
    {"title": "Income___One_Z", "coordinate": [1400, 1284, 1600, 1352]},
    {"title": "Income___Two_B", "coordinate": [1400, 1352, 1600, 1384]},
    {"title": "Income___Three_B", "coordinate": [1400, 1384, 1600, 1417]},
    {"title": "Income___Four_B", "coordinate": [1400, 1417, 1600, 1450]},
    {"title": "Income___Five_B", "coordinate": [1400, 1450, 1600, 1484]},
    {"title": "Income___Six_B", "coordinate": [1400, 1484, 1600, 1515]},
    {"title": "Income___Seven",   "coordinate": [1400, 1515, 1600, 1585]},
    {"title": "Income___Eight",   "coordinate": [1400, 1585, 1600, 1618]},
    {"title": "Income___Nine",   "coordinate": [1400, 1618, 1600, 1650]},
    {"title": "Income___Ten",  "coordinate": [1400, 1650, 1600, 1685]},
    {"title": "Income___Eleven",  "coordinate": [1400, 1680, 1600, 1719]},
    {"title": "Income___Twelve",  "coordinate": [1400, 1715, 1600, 1752]},
    {"title": "Income___Thirteen",  "coordinate": [1400, 1748, 1600, 1785]},
    {"title": "Income___Fourteen",  "coordinate": [1400, 1780, 1600, 1819]},
    {"title": "Income___Fifteen",  "coordinate": [1400, 1814, 1600, 1854]},
]



for data in datas:
    for block in text_blocks:
        for line in block['lines']:
            for span in line['spans']:
                
                x0, y0, x1, y1 =data["coordinate"]
                # x0, y0, x1, y1 =111,329,639,361
                if x0 > x1:
                    x0, x1 = x1, x0
                if y0 > y1:
                    y0, y1 = y1, y0
            draw.rectangle((x0, y0, x1, y1), fill=None, outline='green', width=width)
            # print(x0,y0,x1,y1)

image.show()


In [ ]:
import pandas as pd
df = pd.DataFrame(columns=["file_name",'label', 'value', 'x0', 'y0', 'x2', 'y2'])

In [ ]:
import fitz
import pandas as pd

file_name = "1040_file_180.pdf"
pdf_dir = f"./Data Extraction/Train Files/{file_name}"  # Replace with your PDF file path
doc = fitz.open(pdf_dir)
page = doc[0]

blocks = page.get_text("dict")['blocks']
text_blocks = [block for block in blocks if block['type'] == 0]

def extract_text_from_coordinates(pdf_dir, target_text):
    doc = fitz.open(pdf_dir)
    page = doc[0]
    
    target_x0, target_y0, target_x1, target_y1 = 0, 0, 0, 0
    
    blocks = page.get_text("dict")['blocks']
    text_blocks = [block for block in blocks if block['type'] == 0]

    for block in text_blocks:
        for line in block['lines']:
            for span in line['spans']:
                if target_text in span['text']:
                    x0, y0, x1, y1 = [coord * TARGET_DPI / scaleX for coord in span['bbox']]
                    target_x0, target_y0, target_x1, target_y1 = x0, y0, x1, y1

    return [target_x0, target_y0, target_x1, target_y1]

df = pd.DataFrame(columns=['file_name', 'label', 'value', 'x0', 'y0', 'x2', 'y2'])

for data in datas:
    target_text = None
    for block in text_blocks:
        for line in block['lines']:
            for span in line['spans']:
                x0, y0, x1, y1 = [coord * TARGET_DPI / scaleX for coord in span['bbox']]
                target_x0, target_y0, target_x1, target_y1 = data["coordinate"]
                if target_x0 <= x0 <= target_x1 and target_y0 <= y0 <= target_y1 and \
                        target_x0 <= x1 <= target_x1 and target_y0 <= y1 <= target_y1:
                    target_text = span['text']

    # Print the text at the given coordinates
    if target_text is not None:
        print(f"Text at coordinates ({target_x0}, {target_y0}, {target_x1}, {target_y1}): {target_text}")

        coordinate = extract_text_from_coordinates(pdf_dir=pdf_dir, target_text=target_text)

        new_row = {"file_name": file_name, 'label': data["title"], "value": target_text,
                   'x0': coordinate[0], 'y0': coordinate[1], 'x2': coordinate[2], "y2": coordinate[3]}
        df.loc[len(df)] = new_row 


In [ ]:
df

In [ ]:
import fitz
import pandas as pd
df = pd.DataFrame(columns=['file_name', 'label', 'value', 'x0', 'y0', 'x2', 'y2'])
i=1
while True:
    if i == 201:
        break
    file_name = f"1040_file_{i}.pdf"
    pdf_dir = f"./Data Extraction/Test Files/{file_name}"  
    try:
        doc = fitz.open(pdf_dir)
        page = doc[0]
    except :
        print(f"File not found: {file_name}. Skipping...")
        i += 1
        continue

    blocks = page.get_text("dict")['blocks']
    text_blocks = [block for block in blocks if block['type'] == 0]

    def extract_text_from_coordinates(pdf_dir, target_text):
        doc = fitz.open(pdf_dir)
        page = doc[0]
        
        target_x0, target_y0, target_x1, target_y1 = 0, 0, 0, 0
        
        blocks = page.get_text("dict")['blocks']
        text_blocks = [block for block in blocks if block['type'] == 0]

        for block in text_blocks:
            for line in block['lines']:
                for span in line['spans']:
                    if target_text in span['text']:
                        x0, y0, x1, y1 = [coord * TARGET_DPI / scaleX for coord in span['bbox']]
                        target_x0, target_y0, target_x1, target_y1 = x0, y0, x1, y1

        return [target_x0, target_y0, target_x1, target_y1]
    
    for data in datas:
        target_text = None
        for block in text_blocks:
            for line in block['lines']:
                for span in line['spans']:
                    x0, y0, x1, y1 = [coord * TARGET_DPI / scaleX for coord in span['bbox']]
                    target_x0, target_y0, target_x1, target_y1 = data["coordinate"]
                    if target_x0 <= x0 <= target_x1 and target_y0 <= y0 <= target_y1 and \
                            target_x0 <= x1 <= target_x1 and target_y0 <= y1 <= target_y1:
                        target_text = span['text']

        # Print the text at the given coordinates
        if target_text is not None:
            # print(f"Text at coordinates ({target_x0}, {target_y0}, {target_x1}, {target_y1}): {target_text}")

            coordinate = extract_text_from_coordinates(pdf_dir=pdf_dir, target_text=target_text)

            new_row = {"file_name": file_name, 'label': data["title"], "value": target_text,
                    'x0': coordinate[0], 'y0': coordinate[1], 'x2': coordinate[2], "y2": coordinate[3]}
            df.loc[len(df)] = new_row 
            # print(new_row)
            # print(coordinate)   
    i=i+1

    


pd.set_option('display.max_rows', 200)
df.to_csv("./test.csv")
df

In [ ]:
import fitz
import pandas as pd
df = pd.DataFrame(columns=['file_name', 'label', 'value', 'x0', 'y0', 'x2', 'y2'])
i=1
while True:
    if i == 201:
        break
    file_name = f"1040_file_{i}.pdf"
    pdf_dir = f"./Data Extraction/Train Files/{file_name}"  
    try:
        doc = fitz.open(pdf_dir)
        page = doc[0]
    except :
        print(f"File not found: {file_name}. Skipping...")
        i += 1
        continue

    blocks = page.get_text("dict")['blocks']
    text_blocks = [block for block in blocks if block['type'] == 0]

    def extract_text_from_coordinates(pdf_dir, target_text):
        doc = fitz.open(pdf_dir)
        page = doc[0]
        
        target_x0, target_y0, target_x1, target_y1 = 0, 0, 0, 0
        
        blocks = page.get_text("dict")['blocks']
        text_blocks = [block for block in blocks if block['type'] == 0]

        for block in text_blocks:
            for line in block['lines']:
                for span in line['spans']:
                    if target_text in span['text']:
                        x0, y0, x1, y1 = [coord * TARGET_DPI / scaleX for coord in span['bbox']]
                        target_x0, target_y0, target_x1, target_y1 = x0, y0, x1, y1

        return [target_x0, target_y0, target_x1, target_y1]
    
    for data in datas:
        target_text = None
        for block in text_blocks:
            for line in block['lines']:
                for span in line['spans']:
                    x0, y0, x1, y1 = [coord * TARGET_DPI / scaleX for coord in span['bbox']]
                    target_x0, target_y0, target_x1, target_y1 = data["coordinate"]
                    if target_x0 <= x0 <= target_x1 and target_y0 <= y0 <= target_y1 and \
                            target_x0 <= x1 <= target_x1 and target_y0 <= y1 <= target_y1:
                        target_text = span['text']

        # Print the text at the given coordinates
        if target_text is not None:
            # print(f"Text at coordinates ({target_x0}, {target_y0}, {target_x1}, {target_y1}): {target_text}")

            coordinate = extract_text_from_coordinates(pdf_dir=pdf_dir, target_text=target_text)

            new_row = {"file_name": file_name, 'label': data["title"], "value": target_text,
                    'x0': coordinate[0], 'y0': coordinate[1], 'x2': coordinate[2], "y2": coordinate[3]}
            df.loc[len(df)] = new_row 
    i=i+1

    


pd.set_option('display.max_rows', 200)
df.to_csv("./train.csv")
df